1 - Inventory snapshot info (report)
2 - New Product sales report wmargin
3 - Gsheet with expiry date from CWH
4 - Location based stick info


In [453]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
today = dt.now()
mask = '%Y%m%d'
dte = dt.now().strftime(mask)

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)

#select only required columns

Assortment = Assortment[['createdatl','updatedatl','Getir Product Id','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','transfer_groups','isvisible',\
'isbundle','exp_days_lifetime','exp_days_allowed','exp_days_warning','exp_days_dead','inbox_quantitiy',\
'palet_inbox_quantity','storagetype','transfer_coli_count','is_picked_to_zero','market_status']]

#filter unwanted segments from the dataframe
Segment_filters = ["Asset", "None", "Test", "Only Promo"]
Assortment = Assortment[~Assortment.segment_name.isin(Segment_filters)].reset_index(drop = True)

#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#------------------------------

Path_G = '/Users/swaroopbasavarajmusti/Desktop/current_stock_info.xlsx'
stock_df = pd.read_excel(Path_G)

df = stock_df.copy()

df.rename(columns={'product_id':'Getir Product Id'}, inplace=True)

Closed_Warehouses = ['Alex','Aplerbeck','Barmbek_Gorillas','Berlin HUB','Bickendorf','Billstedt',\
           'Bispingen CW_NoNag','Boeblinger_Gorillas','Cologne - CHOCO','DE_Training_WH','Dortmund HUB - Choco',\
           'Dummy Berlin CW','Friedenau','Gbrunnen Fleet','Getir DE Defleet','Getir DE Office','Getir Germany Test',\
           'Haidhausen','Halensee','Harras','Jungfernheide','JungleTEST','Langwasser','Lichtenberg','Luft_Gorillas',\
           'MagnaPark CW_NoNag','Meringplatz','Munich HUB - Choco','Nordbahnhof','Nürnberg HUB - Choco','Ottensen',\
           'Pasing','Staaken','Stellingen','Tempelhof','Trudering','UNeukolln','Unterbilk','Unterrath','Velodrom',\
           'Wandsbek','Weissensee','Winsviertel','Wittenbergplatz','ZOBMesse','Zabo','Zeigner_Gorillas']

CWH = ['Bispingen CW', 'MagnaPark CW']

for C in Closed_Warehouses:
    df = df[df.warehouse_name != C]

warehouse_list = df.warehouse_name.unique().tolist()
warehouse_list.remove('Bispingen CW')
warehouse_list.remove('MagnaPark CW')

df_1 = df.pivot_table(index=['Getir Product Id'],columns='warehouse_name',values = 'available',aggfunc='sum').\
                reset_index()

#df_1.drop(columns = Closed_Warehouses)

df_1['Total_MFC'] = int(0)
for w in warehouse_list:
    df_1[w]= df_1.fillna(0)[w].astype(int)
    df_1['Total_MFC'] = df_1['Total_MFC'] + df_1[w]
    

df_1['Total_CWH'] = int(0)
for w in CWH:
    df_1[w]= df_1.fillna(0)[w].astype(int)
    df_1['Total_CWH'] = df_1['Total_CWH'] + df_1[w]

df_1['Total'] = int(0)
df_1['Total'] = df_1['Total_MFC'] + df_1['Total_CWH']

Merged_assortment_stock = pd.merge(Assortment, 
                      df_1, 
                      on ='Getir Product Id', 
                      how ='left')

Merged_assortment_stock['Stock_Value_Total'] = Merged_assortment_stock['Total']*\
Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Stock_Value_MFC'] = Merged_assortment_stock['Total_MFC']*\
Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Stock_Value_CWH'] = Merged_assortment_stock['Total_CWH']*\
Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Stock_Value_Total'] = round(Merged_assortment_stock['Stock_Value_Total'].fillna(0).astype(int),2)
Merged_assortment_stock['Stock_Value_MFC'] = Merged_assortment_stock['Stock_Value_MFC'].fillna(0).astype(int)
Merged_assortment_stock['Stock_Value_CWH'] = Merged_assortment_stock['Stock_Value_CWH'].fillna(0).astype(int)
Merged_assortment_stock['Profit/Product'] = Merged_assortment_stock['g10_price']-Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Profit/Product'] = round(Merged_assortment_stock['Profit/Product'],2)
#cat_wise = Merged_assortment_stock.copy()

#cat_wise = cat_wise.groupby(['category_name_en','subcategory_name_en']).sum('Stock_Value').reset_index()
#cat_wise = cat_wise.sort_values('Stock_Value', ascending=False).reset_index(drop = True)

Path_H = '/Users/swaroopbasavarajmusti/Desktop/marketProductSales.xlsx'
Sales_df = pd.read_excel(Path_H)

Sales_df.rename(columns={'product_id':'Getir Product Id'}, inplace=True)
dict_of_Marketplaces = {k: v for k, v in Sales_df.groupby('integrations_breakdown')}
Marketplaces = Sales_df['integrations_breakdown'].unique()
Unique_ID_df = pd.DataFrame(Sales_df['Getir Product Id'].unique())
Unique_ID_df.rename(columns={0:'Getir Product Id'}, inplace=True)

All_means = pd.DataFrame()
for M in Marketplaces:
    df = dict_of_Marketplaces[M].copy()
    df = df.pivot_table(index=['Getir Product Id'],columns='week',values = 'items_sold_count',aggfunc='sum').\
                reset_index()
    temp_name = M +"_mean"
    df[temp_name] = df.mean(numeric_only=True, axis=1)
    df[temp_name] = df[temp_name].astype(int)
    df = df[['Getir Product Id', temp_name ]]
    Unique_ID_df = Unique_ID_df.merge(df, on ='Getir Product Id', how ='left')
    
Master = pd.merge(Merged_assortment_stock, 
                  Unique_ID_df, 
                  on ='Getir Product Id', 
                  how ='left')

import warnings
warnings.simplefilter("ignore")
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file', \
        'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.readonly']

creds = ServiceAccountCredentials.from_json_keyfile_name('panel-321812-ac5afc8690ea.json',scope)
client = gspread.authorize(creds)

Expiry_data  = client.open('Gsheet trial').worksheet('Stock')
Expiry_df = pd.DataFrame(Expiry_data.get_all_records())

Expiry_df['Today'] = dt.now().strftime('%Y%m%d')
Expiry_df['Today'] = pd.to_datetime(Expiry_df['Today'], errors='coerce') 

Expiry_df['Expire date'] = pd.to_datetime(Expiry_df['Expire date'], errors='coerce')

Expiry_df['Remaining_Days'] = Expiry_df['Expire date'] - Expiry_df['Today']
Expiry_df['Remaining_Days'] = pd.to_timedelta(Expiry_df.Remaining_Days, errors='coerce').dt.days

Expiry_df = Expiry_df[['Item code','Expire date','Today','Remaining_Days']]\
.sort_values(by = 'Remaining_Days', ascending=[True]).reset_index(drop = True)
Expiry_df.rename(columns={'Item code':'Getir Product Id'}, inplace=True)

Master_V2 = pd.merge(Master, 
                  Expiry_df, 
                  on ='Getir Product Id', 
                  how ='left')

Master_V2 = Master_V2.drop_duplicates(subset=['Getir Product Id', 'Remaining_Days'])

Analysis = Master_V2.copy()

# Expiry date nearing data

Path_I = '/Users/swaroopbasavarajmusti/Desktop/Location_Based_Stocks.xlsx'
L2_stocks = pd.read_excel(Path_I)

for C in Closed_Warehouses:
    df = L2_stocks[L2_stocks['Warehouse Name'] != C]

df_3 = df.copy()
df_3['expiry_group'] = df_3['Expiry Date'].dt.strftime('%Y')+"_"+df_3['Expiry Date'].dt.strftime('%W')
df_3['Dead_group'] = df_3['Dead Date'].dt.strftime('%Y')+"_"+df_3['Dead Date'].dt.strftime('%W')

# 1st method
df_4 = df_3.pivot_table(index=['Product Id'],columns=['expiry_group','Dead_group','Warehouse Name'],values = 'Quantity',aggfunc='sum', fill_value=0).\
                reset_index()

df_6 = df_4.iloc[:,0:401]

df_6.reset_index(inplace=True)
df_6.columns = [' '.join(col).strip() for col in df_6.columns.values]

# 2nd method
df_5 = df_3.groupby(['expiry_group','Dead_group','Expiry Date','Dead Date','Warehouse Name','Product Id']).agg({"Quantity": "sum"})\
.reset_index().rename(columns={"Quantity": "Remaining_Quantity"})

df_5.rename(columns={'Product Id':'Getir Product Id'}, inplace=True)


expiring_products_data = pd.merge(df_5, 
                      Master, 
                      on ='Getir Product Id', 
                      how ='left')

df_7 = expiring_products_data.copy()
df_7['Remaining_days_expiry'] = df_7['Expiry Date'] - today
df_7['Remaining_days_expiry'] = pd.to_timedelta(df_7.Remaining_days_expiry, errors='coerce').dt.days
df_7['Remaining_days_Dead'] = df_7['Dead Date'] - today
df_7['Remaining_days_Dead'] = pd.to_timedelta(df_7.Remaining_days_Dead, errors='coerce').dt.days
expiring_products_30_days = df_7.loc[df_7['Remaining_days_Dead'] < 31]
expiring_products_30_days = expiring_products_30_days[['expiry_group','Expiry Date','Dead_group','Dead Date','Warehouse Name','Getir Product Id',\
                                                      'Remaining_Quantity','createdatl','product_barcode','marketproduct_name_de',\
                                                      'segment_name','category_name_en','subcategory_name_en','supplier_names',\
                                                      'net_net_buying_price_with_vat','storagetype','Profit/Product','GORILLAS_mean','JET_mean',\
                                                      'GETIR_mean','Remaining_days_Dead','Remaining_days_expiry']]

#Print all data into excel for send out

with pd.ExcelWriter("/Users/swaroopbasavarajmusti/Desktop/Promotional_Products_{}.xlsx".format(dte)) as writer:
    expiring_products_30_days.to_excel(writer, sheet_name = "Products_Expiring_30_days")
    for M in Marketplaces:
        Analysis['Cover_days_'+M] = (Analysis['Total']/Analysis[M+'_mean'])*7
        Analysis['Cover_days_'+M] = Analysis['Cover_days_'+M].fillna(0).astype(int)
        Analysis['STR_'+M] = round((Analysis[M+'_mean']/Analysis['Total'])*100, 2)
        Analysis['Risk_Rank_'+M] = round((Analysis['Cover_days_'+M] - Analysis['Remaining_Days']), 0).rank(ascending=False)
        Analysis['Risk_Rank_'+M] = Analysis['Risk_Rank_'+M].fillna(0).astype(int)
        
        temp_df = Analysis.copy()
        temp_df =  temp_df[(temp_df['Cover_days_'+M]>60)]
        temp_df = temp_df[['createdatl','Getir Product Id','product_barcode','marketproduct_name_de',\
                          'category_name_en','subcategory_name_en','segment_name','Total_CWH','Total_MFC',\
                           'Total','Stock_Value_Total','Stock_Value_MFC','Stock_Value_CWH','Profit/Product',\
                          M+'_mean','STR_'+M,'Cover_days_'+M,'Expire date','Remaining_Days','Risk_Rank_'+M]].sort_values(by="Cover_days_"+M,ascending = False).reset_index(drop = True)
        temp_df = temp_df.sort_values(by='Risk_Rank_'+M,ascending = True).reset_index(drop = True)
        temp_df.to_excel(writer, sheet_name = M)
        col = "Risk_Rank_"+M
        temp_df_2 = temp_df[(temp_df[col] > 0)& (temp_df[col] < 31)].reset_index(drop = True)
        temp_df_2.to_excel(writer, sheet_name = "top_30_"+M)



In [446]:
#df_5.to_excel("/Users/swaroopbasavarajmusti/Desktop/df_5.xlsx")